In [1]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [2]:
#zero initialization
neuron = Neuron(2,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [3]:
#random initialization
neuron = Neuron(2,identity,method_ini = "RandomNor", random_mean = -1000.0, random_std = 10.0)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [4]:
#initialization from list
vector = [1.0,2.0,3.5]
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [5]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
neuron = Neuron(vector,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [6]:
#initialization from row vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_row = vector[np.newaxis,:]
neuron = Neuron(vector_row,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [7]:
#initialization from column vector(2D array)
vector = np.array([1.0,2.0,3.5])
vector_column = vector[:,np.newaxis]
neuron = Neuron(vector_column,identity)
#forward pass test
testInputFormat([1.0,1.0],neuron)

True

In [8]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="RandomNor")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: no function given as activation function
Correct errors was caught. Error: Given input is not supported by implementation for activation function initialization. Given variable is not a function and thus can not be used as activation function.
Test for: not vector given for weights initialization (2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (not column or row vector in 2 dim), so it is unsuitable for neuron weight initialization.
Test for: not vector given for weights initialization (higher than 2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (due to more than 2 dim), so it is unsuitable for neuron weight initialization.
Test for: wrong weights given for initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization.

Layer class implementation testing

In [9]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [10]:
#initialization by number of dim in list
dim_layer = [4,5]
testLayer = Layer(dim_layer,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [11]:
#initialization by giving weight array
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,identity,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [12]:
#initialization by giving weight array and function (single) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
testLayer = Layer(weights,[identity],method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [13]:
#initialization by giving weight array and identity function (mutliple same) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [identity,identity,identity]
testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [14]:
#initialization by giving weight array and identity function (mutliple different) from list
weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
activation_functions = [sigmoid,relu,leaky_relu]
testLayer = Layer(weights,activation_functions)
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testLayer)

True

In [15]:
#error messages testing
print("Test for: wrong var type (not number) given in list for weight initialization")
try:
    vector = ["as","2.0"]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in list for weight initialization")
try:
    vector = [1,2.3]
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type (not number) given in 1D array for weight initialization")
try:
    vector = np.array(["as","2.0"])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given (number, but no int) in 1D array for weight initialization")
try:
    vector = np.array([1,2.3])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong dimensions of given 1D array for weight initialization")
try:
    vector = np.array([1,2,1,1,2])
    testLayer = Layer(vector,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not rational numbers given in 2D array for weight initialization")
try:
    weights = np.array([[1.0 + 4j,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough information given in 2D array for weight initialization")
try:
    weights = np.array([[]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: bigger than 2D was given for array for weight initialization")
try:
    weights = np.array([[[2]]])
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not supported data type given for array for weight initialization")
try:
    weights = "hehe"
    testLayer = Layer(weights,identity,method_ini="RandomNor")   
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not compatible amount of activation functions given to the amount of neurons for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,relu,leaky_relu,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (inside list) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,1,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not activation function given (standalone) for activation function initialization")
try:
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = 1
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, too much dimensions in input")
try:
    #Input data for input propagation (forward pass)
    inputTestArray = np.array([[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]],[[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, complex input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = np.array([[1.0 + 3j],[1.0],[1.0],[1.0],[1.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input")
try:
    #Input data for input propagation (forward pass)
    inputTestVect = "sda"
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestVect)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, wrong data type of input in list")
try:
    #Input data for input propagation (forward pass)
    inputTestList =["a",1.0,1.0,1.0,1.0]
    inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
    testLayer.forward(inputTestArray)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation, input does not match layer neurons")
try:
    #Input data for input propagation (forward pass)
    inputTestList = [1.0,1.0,1.0,1.0,1.0,1.0,1.0]
    #model ini
    weights = np.array([[1.0,-4.0,-4.0,3.0,3.0,3.0],[-1.0,-4.0,3.5,3.5,3.5,3.0],[2.0,-4.0,-2.5,3.0,2.2,3.0]])
    activation_functions = [sigmoid,sigmoid,leaky_relu]
    testLayer = Layer(weights,activation_functions,method_ini="RandomNor")
    #forward pass test
    testLayer.forward(inputTestList)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: wrong var type (not number) given in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in list for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type (not number) given in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given values are not integers and thus can not represent of layer weights matrix.
Test for: wrong var type given (number, but no int) in 1D array for weight initialization
Correct errors was caught. Error: Given input is not supported by implementation for weight

FNN class implementation testing

In [16]:
#initialization by number of dim in vector(list)
dim_layer = [5,5,3,2,1]
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list_list[0])
print(testNet.activ_functions_list_list[1])
print(testNet.activ_functions_list_list[2])
print(testNet.activ_functions_list_list[3])
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[[ 0.72865573 -0.69064015  0.20781378 -0.12664006 -1.45048394 -0.24953167]
 [ 0.09156754  1.46336604  0.64830407 -1.63432636  1.02226421 -0.00653283]
 [ 2.33725045  0.38244568  0.37070914  2.54040866  0.12930808  0.43994684]
 [ 1.24098007 -1.15614916 -0.80953651  0.69900978  0.96561623  1.41925041]
 [ 0.73248021 -1.21443568  0.08598795 -0.18773519 -0.5016249   0.39213141]]
[[-1.15027946 -0.30735672  0.36868503 -0.70253999 -1.26867975  0.85282615]
 [ 0.12064178 -0.73489035  0.20607762  1.63664024  0.68764285  3.29425817]
 [-0.20076453 -0.53785428 -0.68498549  0.01374294 -0.0389834   1.22773857]]
[[-0.36974909  0.84623864  1.26990404  0.68487164]
 [ 0.89420701  0.29297234  0.59991213 -1.04811846]]
[[1.11802256 1.16042742 0.76556817]]
[<function relu at 0x0000014EFE3BEF20>, <function relu at 0x0000014EFE3BEF20>, <function relu at 0x0000014EFE3BEF20>, <function relu at 0x0000014EFE3BEF20>, <function relu at 0x0000014EFE3BEF20>]
[<function relu at 0x0000014EFE3BEF20>, <function relu at 0x00

True

In [17]:
#initialization by number of dim in vector(1D array)
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [18]:
#dummy network ini
dim_layer = np.array([5,5,3,2,1])
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#initialization by ready weight array
dim_layer = testNet.weights_list
print(dim_layer)
activ_func = [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

[array([[-0.70818547, -1.00849993,  0.72195795,  1.28164888, -0.48274614,
         0.82363815],
       [ 0.68545155, -1.7818595 , -0.9901896 ,  0.043508  ,  1.49372644,
        -0.13111481],
       [ 0.48811464, -0.80679127, -1.27109505, -0.46066746,  1.55550587,
        -0.49232216],
       [-1.43625796, -0.41236175, -1.70676629,  1.55133565, -1.24594354,
        -1.85875151],
       [ 0.85953661,  1.23456248,  0.71006924,  0.0768556 ,  0.89634282,
        -0.37313133]]), array([[-0.54957204,  0.4525584 , -1.54867478,  1.09607812, -0.2694027 ,
         3.65520084],
       [ 0.1214031 ,  0.22300076,  1.10650272, -1.62669043,  2.09995229,
        -0.06120554],
       [-0.3462986 ,  0.76764967,  1.7191373 , -0.65090686,  0.71762318,
        -0.24095128]]), array([[-1.16791795, -1.36362139,  0.3167629 , -0.98286643],
       [ 0.66202682,  0.08010841,  0.12422215, -0.60428016]]), array([[-0.06882011, -0.07727074,  0.0534936 ]])]


True

In [19]:
#initialization by number of dim in vector(1D array) and 1 activation function(as value)
dim_layer = np.array([5,5,3,2,1])
activ_func = identity
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [20]:
#initialization by number of dim in vector(1D array) and 1 activation function(as list)
dim_layer = np.array([5,5,3,2,1])
activ_func = [identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [21]:
#initialization by number of dim in vector(1D array) and 2 activation functions
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [22]:
#initialization by number of dim in vector(1D array) and activation function for each layer
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,sigmoid,leaky_relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [23]:
#initialization by number of dim in vector(1D array) and activation function for each neuron
dim_layer = np.array([5,5,3,2,1])
l1 = [relu,sigmoid,sigmoid,leaky_relu,identity]
l2 = [relu,sigmoid,sigmoid]
l3 = [relu,sigmoid]
l4 = [identity]
activ_func =  [l1,l2,l3,l4]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [24]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Zero weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="Zero")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [25]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [26]:
#initialization by number of dim in vector(1D array) and 2 activation functions with random normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="RandomNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [27]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [28]:
#initialization by number of dim in vector(1D array) and 2 activation functions with Xavier normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="XavNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [29]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He uniform weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeUni")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [30]:
#initialization by number of dim in vector(1D array) and 2 activation functions with He normalized weight initialization
dim_layer = np.array([5,5,3,2,1])
activ_func =  [relu,identity]
testNet = FNN(dim_layer,activ_func,method_ini="HeNor")
#testing forward pass
testInputFormat([1.0,5.4,-1.0,21.0,-1.0],testNet)

True

In [31]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="RandomNor")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: not supported input for weights
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Not supported data type given.
Test for: array of not supported dimension
Correct errors was caught. Error:  Given array is of size not supported by implementation. Supported dimensions: 1.
Test for: given numbers are not integers
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Given values in vector must be integers to properly represent number of neurons in layers
Test for: not enough numbers given. Minimally required input and output layers cannot be created
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. At least two numbers are needed for FNN creation: FNN needs at least input and output layer (single layer network).


Reload Modules


In [32]:
import importlib
import sys
import numpy as np

# Reload order:
modules_in_order = [
    "ActivFunctions",
    "SuppFunctions",
    "InitFunctions",
    "Layer",
    "FNN",
    "TrainingFunctions"
]

for m in modules_in_order:
    if m in sys.modules:
        importlib.reload(sys.modules[m])
    else:
        globals()[m] = importlib.import_module(m)

print("All modules reloaded successfully.")


All modules reloaded successfully.


Imports

In [33]:
from FNN import FNN
from TrainingFunctions import backwards
from LossFunctions import MeanSquaredErrorDerivative

# Activations
from ActivFunctions import identity, sigmoid, tanh, relu, leaky_relu
import numpy as np


In [34]:
print("=== Test 1: Single-Layer Network Gradient ===")

dim_layer = np.array([4, 5])   # 4 inputs -> 5 outputs
testNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputTestVect = np.ones((4, 1))
targetVect = 0.5 * np.ones((5, 1))

grads = backwards(testNet, inputTestVect, targetVect, MeanSquaredErrorDerivative)

print("\nGradient matrix for single layer:")
print(grads[0])


=== Test 1: Single-Layer Network Gradient ===

Gradient matrix for single layer:
[[-0.0648399  -0.0648399  -0.0648399  -0.0648399  -0.0648399 ]
 [ 0.01855704  0.01855704  0.01855704  0.01855704  0.01855704]
 [ 0.07876653  0.07876653  0.07876653  0.07876653  0.07876653]
 [-0.28770093 -0.28770093 -0.28770093 -0.28770093 -0.28770093]
 [-0.32630573 -0.32630573 -0.32630573 -0.32630573 -0.32630573]]


In [35]:
print("=== Test 2: Deep 10-Layer Network ===")

dim_layer = np.array([
    5, 8, 6, 7, 5, 9, 4, 3, 2, 4, 1
])

deepNet = FNN(dim_layer, identity, method_ini="RandomNor")

inputVect = np.ones((5, 1))
targetVect = np.array([[0.7]])

grads = backwards(deepNet, inputVect, targetVect, MeanSquaredErrorDerivative)

for i, g in enumerate(grads):
    print(f"\nGradient matrix for layer {i}:")
    print(g)


=== Test 2: Deep 10-Layer Network ===

Gradient matrix for layer 0:
[[ 0.05202706  0.05202706  0.05202706  0.05202706  0.05202706  0.05202706]
 [-0.13912258 -0.13912258 -0.13912258 -0.13912258 -0.13912258 -0.13912258]
 [ 0.10714015  0.10714015  0.10714015  0.10714015  0.10714015  0.10714015]
 [ 0.18328649  0.18328649  0.18328649  0.18328649  0.18328649  0.18328649]
 [ 0.18791179  0.18791179  0.18791179  0.18791179  0.18791179  0.18791179]
 [-0.16812287 -0.16812287 -0.16812287 -0.16812287 -0.16812287 -0.16812287]
 [ 0.13727115  0.13727115  0.13727115  0.13727115  0.13727115  0.13727115]
 [ 0.130814    0.130814    0.130814    0.130814    0.130814    0.130814  ]]

Gradient matrix for layer 1:
[[ 4.13208991e-03 -5.61097656e-03  8.31281466e-03  1.40299743e-02
   5.27890001e-04  2.85357526e-03 -5.31270856e-03  1.23083273e-02
   4.45842830e-03]
 [ 1.56337820e-01 -2.12291568e-01  3.14515742e-01  5.30824753e-01
   1.99727436e-02  1.07965157e-01 -2.01006584e-01  4.65686156e-01
   1.68684849e-01]